In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
# df_train = df_train.iloc[:-686]

In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

27

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
# df_latest[df_latest['player'] == 'Chris Wood']
# lucas digne
# chris wood

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [11]:
last_play_proba.head()

,player,team,play_proba
0,Theo Walcott,Southampton,1.0
1,Fraser Forster,Southampton,1.0
2,Alex McCarthy,Southampton,0.0
3,Oriol Romeu Vidal,Southampton,1.0
4,Nathan Redmond,Southampton,1.0


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [13]:
df_latest.drop_duplicates(inplace=True)

In [14]:
df_train.drop_duplicates(inplace=True)

In [15]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [16]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [17]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [18]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
Bernardo Silva
João Cancelo
Rúben Dias
Rodri
Aymeric Laporte
Raheem Sterling
Phil Foden
Kyle Walker
Kevin De Bruyne
İlkay Gündoğan
Gabriel Jesus
Jack Grealish
Riyad Mahrez
John Stones
Nathan Aké
Fernandinho
Oleksandr Zinchenko
Ferrán Torres
Cole Palmer
Zack Steffen
Benjamin Mendy
James Mcatee
Liam Delap
Kayky Chagas
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Alisson
Virgil van Dijk
Mohamed Salah
Trent Alexander-Arnold
Sadio Mané
Joël Matip
Jordan Henderson
Andrew Robertson
Diogo Jota
Fabinho
Thiago Alcántara
Alex Oxlade-Chamberlain
Roberto Firmino
Naby Keïta
Kostas Tsimikas
Curtis Jones
Ibrahima Konaté
James Milner
Harvey Elliott
Luis Díaz
Caoimhín Kelleher
Joe Gomez
Tyler Morton
Takumi Minamino
Divock Origi
Kaide Gordon
Neco Williams
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Antonio Rüdiger
Edouard Mendy
Thi

João Pedro
Cucho
Jeremy Ngakia
Francisco Sierralta
Edo Kayembe
Ken Sema
Oghenekaro Etebo
Ozan Tufan
Christian Kabasele
Nicolas Nkoulou
Ashley Fletcher
Troy Deeney
Dan Gosling
Norwich City
https://fbref.com/en/squads/1c781004/Norwich-City-Stats
Max Aarons
Teemu Pukki
Kenny McLean
Ben Gibson
Grant Hanley
Tim Krul
Brandon Williams
Pierre Lees-Melou
Milot Rashica
Billy Gilmour
Josh Sargent
Mathias Normann
Ozan Kabak
Angus Gunn
Dimitris Giannoulis
Adam Idah
Przemysław Płacheta
Lukas Rupp
Todd Cantwell
Andrew Omobamidele
Jakob Sørensen
Kieran Dowell
Christos Tzolis
Sam Byram
Jon Rowe
Bali Mumba


In [19]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [20]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [31]:
# df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['team'].str[-2:] != 'on'].index]
# df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['squad'] != 'CSKA Moscow'].index]

In [25]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [26]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [27]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7248,2021-2022,Jon Rowe,2021-12-28,Norwich,Premier League,20,0,0,9,5,...,0.1,1,0,3,5,60.0,7,0,1,Jonathan Rowe
7249,2021-2022,Jon Rowe,2022-01-12,Norwich,Premier League,18,0,0,6,4,...,0.0,1,0,1,2,50.0,3,1,1,Jonathan Rowe
7250,2021-2022,Jon Rowe,2022-02-09,Norwich,Premier League,24,0,0,5,7,...,0.0,0,0,1,1,100.0,1,0,0,Jonathan Rowe
7251,2021-2022,Jon Rowe,2022-02-19,Norwich,Premier League,26,0,0,2,0,...,0.0,0,0,1,2,50.0,3,0,0,Jonathan Rowe


In [28]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [29]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [32]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [29]:
# any additional corrections
# match_list['Kayky Chagas'] = 'Kayky da Silva Chagas'
# match_list['Josh Dasilva'] = 'Pelenda Joshua Dasilva'
# match_list['Bruno Guimarães'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Samir Santos'] = 'Samir Caetano de Souza Santos'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

# match_list['Sadio Man�'] = 'Sadio Mané'
# match_list['Jo�l Matip'] = 'Joel Matip'
# match_list['Thiago Alc�ntara'] = 'Thiago Alcántara do Nascimento'
# match_list['Naby Ke�ta'] = 'Naby Keita'
# match_list['Luis D�az'] = 'Luis Díaz'
# match_list['Jos� S�'] = 'José Malheiro de Sá'
# match_list['Romain Sa�ss'] = 'Romain Saïss'
# match_list['N�lson Semedo'] = 'Nélson Cabral Semedo'
# match_list['Ra�l Jim�nez'] = 'Raúl Jiménez'
# match_list['Jo�o Moutinho'] = 'João Filipe Iria Santos Moutinho'
# match_list['R�ben Neves'] = 'Rúben Diogo da Silva Neves'
# match_list['Rayan A�t Nouri'] = 'Rayan Ait Nouri'
# match_list['Fernando Mar�al'] = 'Fernando Marçal'
# match_list['Emiliano Mart�nez'] = 'Emiliano Martínez'
# match_list['Emi Buend�a'] = 'Emiliano Buendía Stati'
# match_list['Fabian Sch�r'] = 'Fabian Schär'
# match_list['Martin D�bravka'] = 'Martin Dubravka'
# match_list['Miguel Almir�n'] = 'Miguel Almirón'
# match_list['Bruno Guimar�es'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Kiko Femen�a'] = 'Francisco Femenía Far'

In [30]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [31]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [ ]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [41]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7248,2021-2022,Jon Rowe,2021-12-28,Norwich,Premier League,20,0,0,9,5,...,0.1,1,0,3,5,60.0,7,0,1,Jonathan Rowe
7249,2021-2022,Jon Rowe,2022-01-12,Norwich,Premier League,18,0,0,6,4,...,0.0,1,0,1,2,50.0,3,1,1,Jonathan Rowe
7250,2021-2022,Jon Rowe,2022-02-09,Norwich,Premier League,24,0,0,5,7,...,0.0,0,0,1,1,100.0,1,0,0,Jonathan Rowe
7251,2021-2022,Jon Rowe,2022-02-19,Norwich,Premier League,26,0,0,2,0,...,0.0,0,0,1,2,50.0,3,0,0,Jonathan Rowe


In [42]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [43]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
128541,Harry Kane,24,4,90,Tottenham Hotspur,Southampton,NaN,NaN,True,2,...,0.5,4.0,1.0,18.0,30.0,60.0,20.0,2.0,2.0,1
129202,Harry Kane,25,4,90,Tottenham Hotspur,Wolverhampton Wanderers,NaN,NaN,True,2,...,0.1,2.0,0.0,16.0,26.0,61.5,33.0,2.0,4.0,1
129993,Harry Kane,26,4,90,Tottenham Hotspur,Manchester City,NaN,NaN,False,13,...,0.1,3.0,1.0,20.0,30.0,66.7,24.0,0.0,0.0,1
129994,Harry Kane,26,4,90,Tottenham Hotspur,Burnley,NaN,NaN,False,2,...,0.1,4.0,0.0,22.0,36.0,61.1,36.0,3.0,3.0,1
130880,Harry Kane,27,4,90,Tottenham Hotspur,Leeds,NaN,NaN,False,11,...,0.8,5,1,25,34,73.5,34,3,5,1


In [44]:
df_train_new.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
131286,Wout Weghorst,27,4,90,Burnley,Leicester City,NaN,NaN,True,1,...,0.0,0,0,18,28,64.3,19,1,1,1
131287,Yan Valery,27,2,0,Southampton,Norwich,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
131288,Yerry Mina,27,2,0,Everton,Manchester City,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
131289,Yerson Mosquera Valdelamar,27,2,0,Wolverhampton Wanderers,West Ham United,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
131290,Yoane Wissa,27,3,61,Brentford,Newcastle United,NaN,NaN,True,2,...,0.0,1,0,11,16,68.8,19,1,1,1
131291,Youri Tielemans,27,3,71,Leicester City,Burnley,NaN,NaN,False,3,...,0.1,3,0,20,27,74.1,19,0,0,1
131292,Yves Bissouma,27,3,90,Brighton and Hove Albion,Aston Villa,NaN,NaN,True,1,...,0.0,1,0,49,52,94.2,42,1,1,1
131293,Zack Steffen,27,1,0,Manchester City,Everton,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
131294,Álvaro Fernández,27,1,0,Brentford,Newcastle United,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
131295,Çaglar Söyüncü,27,2,90,Leicester City,Burnley,NaN,NaN,False,7,...,0.0,2,0,41,55,74.5,27,1,1,1


In [45]:
df_latest_further.shape

(632, 57)

In [46]:
df_train_new.shape

(131296, 59)

In [47]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [48]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [49]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
270,Leicester City,Leeds,2022-03-05T12:30:00Z,28.0,0
283,Leeds,Aston Villa,2022-03-10T19:45:00Z,28.0,1
282,Wolverhampton Wanderers,Watford,2022-03-10T19:30:00Z,28.0,2
281,Southampton,Newcastle United,2022-03-10T19:30:00Z,28.0,3
280,Norwich,Chelsea,2022-03-10T19:30:00Z,28.0,4
278,Manchester City,Manchester United,2022-03-06T16:30:00Z,28.0,5
277,Watford,Arsenal,2022-03-06T14:00:00Z,28.0,6
279,Tottenham Hotspur,Everton,2022-03-07T20:00:00Z,28.0,7
275,Wolverhampton Wanderers,Crystal Palace,2022-03-05T15:00:00Z,28.0,8
274,Norwich,Brentford,2022-03-05T15:00:00Z,28.0,9


In [50]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [51]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,13,2
1,28.0,1,Leeds,Aston Villa,2022-03-10T19:45:00Z,2,7
2,28.0,2,Wolverhampton Wanderers,Watford,2022-03-10T19:30:00Z,39,57
3,28.0,3,Southampton,Newcastle United,2022-03-10T19:30:00Z,20,4
4,28.0,4,Norwich,Chelsea,2022-03-10T19:30:00Z,45,8


In [52]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [53]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
3,Pierre-Emerick Aubameyang,0,4,4,96,3,Aubameyang
7,Pablo Marí,0,2,8,42,3,Marí
8,Héctor Bellerín,0,2,9,48,3,Bellerín
...,...,...,...,...,...,...,...
704,Christian Marques,0,2,549,40,39,Marques
706,Louie Moulden,0,1,611,40,39,Moulden
709,Bruno André Cavaco Jordão,0,3,661,45,39,Jordão
711,Dion Sanderson,0,2,686,40,39,Sanderson


In [54]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [55]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [56]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,28.0,0,Leeds,Leicester City,2022-03-05T12:30:00Z,Pablo Hernández Domínguez,3,55,0,Hernández,False
1,28.0,0,Leeds,Leicester City,2022-03-05T12:30:00Z,Francisco Casilla Cortés,1,45,0,Casilla,False
2,28.0,0,Leeds,Leicester City,2022-03-05T12:30:00Z,Liam Cooper,2,44,0,Cooper,False
3,28.0,0,Leeds,Leicester City,2022-03-05T12:30:00Z,Luke Ayling,2,43,100,Ayling,False
4,28.0,0,Leeds,Leicester City,2022-03-05T12:30:00Z,Mateusz Klich,3,52,100,Klich,False


In [57]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Kasper Schmeichel,1,48,100,Schmeichel,True
1,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Jonny Evans,2,54,0,Evans,True
2,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Marc Albrighton,3,52,100,Albrighton,True
3,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Nampalys Mendy,3,43,100,Mendy,True
4,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Danny Ward,1,40,100,Ward,True


In [58]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [59]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [60]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Kasper Schmeichel,1,4.8,1.00,Schmeichel,True,NaN,NaN,2122
1,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Jonny Evans,2,5.4,0.00,Evans,True,NaN,NaN,2122
2,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Marc Albrighton,3,5.2,1.00,Albrighton,True,NaN,NaN,2122
3,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Nampalys Mendy,3,4.3,1.00,Mendy,True,NaN,NaN,2122
4,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Danny Ward,1,4.0,1.00,Ward,True,NaN,NaN,2122
5,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Jamie Vardy,4,10.3,1.00,Vardy,True,NaN,NaN,2122
6,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Dennis Praet,3,4.9,0.00,Praet,True,NaN,NaN,2122
7,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Ricardo Domingos Barbosa Pereira,2,5.2,0.50,Pereira,True,NaN,NaN,2122
8,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Daniel Amartey,2,3.8,1.00,Amartey,True,NaN,NaN,2122
9,28.0,0,Leicester City,Leeds,2022-03-05T12:30:00Z,Timothy Castagne,2,5.3,0.00,Castagne,True,NaN,NaN,2122


In [61]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')